In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-133298")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-133298
Azure region: southcentralus
Subscription id: 48a74bb7-9950-4cc1-9caa-5d50f995cc55
Resource group: aml-quickstarts-133298


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "cluster1"
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
cluster = ComputeTarget.create(ws, cluster_name, compute_config)
cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
os.listdir()

['.ipynb_checkpoints',
 'outputs',
 'train.py',
 'udacity-project (1).ipynb',
 'Users']

In [9]:
from azureml.core.environment import Environment
Environment(name="myen")

In [31]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler

d = {"--c": choice(1,2), "--max_iter": choice(100,200,300)}
ps = RandomParameterSampling(d)

# Specify a Policy
policy = BanditPolicy (evaluation_interval= 2, slack_factor= 0.1)

# Create a SKLearn estimator for use with train.py
est=SKLearn(source_directory=os.path.join('./'), entry_script='train.py', compute_target=cluster)#, vm_size='STANDARD_D2_V2',vm_priority='lowpriority')
#est = ScriptRunConfig(source_directory='./', script='train.py', compute_target=cluster, environment="myen")
#est._compute_target=cluster

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig (hyperparameter_sampling = ps, primary_metric_name = 'Accuracy', primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, policy = policy, max_total_runs=15, max_concurrent_runs = 4, estimator = est)
                                   

In [32]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_7927d1ad-29b3-4742-9861-a2e1a1bc222b
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_7927d1ad-29b3-4742-9861-a2e1a1bc222b?wsid=/subscriptions/48a74bb7-9950-4cc1-9caa-5d50f995cc55/resourcegroups/aml-quickstarts-133298/workspaces/quick-starts-ws-133298

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-04T13:12:06.942315][API][INFO]Experiment created<END>\n""<START>[2021-01-04T13:12:07.423320][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-04T13:12:07.741367][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-01-04T13:12:08.1776542Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_7927d1ad-29b3-4742-9861-a2e1a1bc222b
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_7927d1ad-29b3-4742-9861-a2e1a1bc222b?wsid=/subscriptions/48a74

{'runId': 'HD_7927d1ad-29b3-4742-9861-a2e1a1bc222b',
 'target': 'cluster1',
 'status': 'Canceled',
 'startTimeUtc': '2021-01-04T13:12:06.760495Z',
 'endTimeUtc': '2021-01-04T13:20:15.425692Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name": "Accur

In [21]:
# Get your best run and save the model from that run.

import joblib
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())

model = best_run.register_model(model_name='Bank_marketing_model_hyperdrive', model_path='./')

AttributeError: 'NoneType' object has no attribute 'get_details'

In [16]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path_url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(path = path_url)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

auto_ml_run = experiment.submit(config = automl_config, show_output = True)
RunDetails(auto_ml_run).show()

In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model = auto_ml_run.get_output()
print(best_run)
print(fitted_model)

model_ml = best_run.register_model(model_name='Bank_marketing_model_auto_ml', model_path='./')